# Diameter reconstruction

Estimating diameters of image data can be challenging. In the [example data directory](../radii/data/neuron1), we provide image data of L5PTs. These neurons have a long and thick dendrite oriinating from the apiex of the soma, aptly called the "apical" dendrite. The slices are cut perpendicular to this dendrite, making it hard to estimate its diameter: the difference between a corkscrewing apical dendrite and a straight and thick apical dendrite is virtually indistinguishable for many conventional microscope resolutions. The other dendrites aren't necessarily easy to reconstruct with diameters either.

This example notebook will guide the user through the module [dendrite_thickness](../../../dendrite_thickness/__init__.py). This module uses a rayburst algorithm to estimate the thickness of the dendrites at each location.

In [1]:
import Interface as I

from getting_started import getting_started_dir
current_dir = I.os.getcwd()
DATA_DIR = I.os.path.join(getting_started_dir, "radii", "data", "neuron1")

am_folder_path = I.os.path.join(DATA_DIR, 'am')
tif_folder_path = I.os.path.join(DATA_DIR, 'tif', 'max_z_projections')
hoc_file_path = I.os.path.join(DATA_DIR, 'hoc', '500_GP_WR639_cell_1547_SP5C_checked_RE.hoc')
output_folder_path = I.os.path.join(DATA_DIR, 'output')
bijective_points_path = I.os.path.join(DATA_DIR, 'landmark', 'am2_transformed_landmark.landmarkAscii')

trying to connect to distributed locking server {'config': {'hosts': 'somalogin02-hs:33333'}, 'type': 'zookeeper'}
success!
[INFO] ISF: Current version: heads/publish+0.g672bbea4.dirty
[INFO] ISF: Current pid: 259164
[INFO] mechanisms: Loading mechanisms:
[WARNING] ISF: Could not load linked views



[INFO] ISF: Loaded modules with __version__ attribute are:
IPython: 7.19.0, Interface: heads/publish+0.g672bbea4.dirty, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, attr: 20.3.0, backcall: 0.2.0, blake3: 0.3.3, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dash: 2.9.3, dask: 2.30.0, dateutil: 2.8.1, deap: 1.3, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, filelock: 3.0.12, flask: 1.1.2, fsspec: 0.8.3, future: 0.18.2, gevent: 20.9.0, greenlet: 0.4.17, ipaddress: 1.0, ipykernel: 5.3.4, ipython_ge

# Image data

Let's start by loading in some image scan from the microscope and see what it looks like.

In [2]:
import glob
image_paths = glob.glob(I.os.path.join(tif_folder_path, '*.tif'))

In [3]:
def downsample(img, factor):
    x, y = I.np.shape(img)
    cut_x, cut_y = x % factor, y % factor
    if cut_x:
        img = img[:-cut_x]
    if cut_y:
        img = img[:,:-cut_y]
    small_image = img.reshape(
        (x//factor, factor, y//factor, factor)).max(3).max(1)
    return small_image

In [4]:
from PIL import Image
downsample_factor = 10  # downsample by quite a lot for image stack
downsampled_image = downsample(I.np.array(Image.open(image_paths[20])), factor=downsample_factor)
I.plt.matshow(downsampled_image, cmap="Greys_r")

What does the entire image stack look like?

In [5]:
from skimage import io
import plotly.graph_objects as go

def get_image_stack(image_paths, z_coordinates, title=''):
    """
    Given an array of image paths and z coordinates, construct a plotly.Figure() object that you can slide through
    based on: https://plotly.com/python/visualizing-mri-volume-slices/ 
    by Emilia Petrisor
        X: @mathinpython
        Github: empet
    
    Args:
        image_paths ([str]): array of image paths
        z_coordinates ([float|int]): array of z coordinates
    """
    vol = [downsample(io.imread(image), 10) for image in image_paths]
    zs = [float(e.split(I.os.sep)[-1][1:3]) for e in image_paths]
    r, c = vol[0].shape
    nb_frames = len(vol)

    fig = go.Figure(
        frames=[go.Frame(
            data=go.Surface(
                z=zs[k] * I.np.ones((r, c)),
                surfacecolor=I.np.flipud(vol[nb_frames - k - 1])),
            name=str(k))
        for k in range(nb_frames)])

    # Add data to be displayed before animation starts
    fig.add_trace(go.Surface(
        z=zs[0] * I.np.ones((r, c)),
        surfacecolor=I.np.flipud(vol[0]),
        colorscale='Greys_r',
        colorbar=dict(thickness=20, ticklen=4)))


    def frame_args(duration):
        return {
                "frame": {"duration": duration},
                "mode": "immediate",
                "fromcurrent": True,
                "transition": {"duration": duration, "easing": "linear"}}

    sliders = [
                {
                    "pad": {"b": 10, "t": 60},
                    "len": 0.9,
                    "x": 0.1,
                    "y": 0,
                    "steps": [
                        {
                            "args": [[f.name], frame_args(0)],
                            "label": str(k),
                            "method": "animate",
                        } for k, f in enumerate(fig.frames)]}]

    # Layout
    fig.update_layout(
             title=title,
             width=600,
             height=600,
             scene=dict(
                        zaxis=dict(range=[min(zs), max(zs)], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1)),
             updatemenus = [
                {
                    "buttons": [
                        {
                            "args": [None, frame_args(50)],
                            "label": "&#9654;", # play symbol
                            "method": "animate"},
                        {
                            "args": [[None], frame_args(0)],
                            "label": "&#9724;", # pause symbol
                            "method": "animate"}],
                    "direction": "left",
                    "pad": {"r": 10, "t": 70},
                    "type": "buttons",
                    "x": 0.1,
                    "y": 0}],
             sliders=sliders)
    return fig

In [6]:
%matplotlib inline
image_paths.sort()
cut_heights = [float(e.split(I.os.sep)[-1][1:3]) for e in image_paths]  # image paths begin with Sxx, where xx is the z-co

fig = get_image_stack(
    image_paths=image_paths, 
    z_coordinates=cut_heights,
    title='Z-projected images of $50 \mu m$ thick Wistar rat barrel cortex slices')
fig.write_html(I.os.path.join(current_dir, 'static', 'slices.html'), auto_open=True)

 This writes out a `.html` file that you can open in your browser, or a separate tab in Jupyter Lab. Depending on which program you're running, you can also render it inline in the notebook, although you may have to override the default plotly renderer for that. Check available renderers with:

In [7]:
import plotly.io as pio
pio.renderers

Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

We created this notebook with the Jupyter server running on a remote machine, so the renderer is set to "iframe_connected". You should adapt this value depending on your Jupyter setup, and IDE (if you're using one).

In [8]:
plotly_renderer = "iframe_connected"

In [9]:
# iframe_connected is recommended if your jupyter server is
# not running on the same machine as the program rendering the data
fig.show(renderer=plotly_renderer)

# Imaged slices to morphological reconstruction 

Making a morphological reconstruction from this slice data can be done in two consecutive processes:
1. Creating a skeleton of connected dendrites form the slice data
2. Assigning diameters to this skeletonized reconstruction

The entire pipeline that takes care of both these processes is conveniently bundled under [`thickness.pipeline`](../../../dendrite_thickness/thickness/pipeline.py)

We will first run the pipeline in its entirety, and then highlight key steps and how they work one by one.

## 1. Image slice to skeleton

There is a variety of deprojection algorithms that take in a 2D image and deproject it onto a skeleton. We used the Amira extension "The Filament Editor" [(Dercksen et al., 2014)](https://link.springer.com/article/10.1007/s12021-013-9213-2#citeas).

<center><img src=./static/Dercksen.webp></center>

Connecting edges at a branching point. **a** Axonal fragments (black arrow), whose 3D configuration is difficult to identify from the 2D MIP. **b** Even when displaying the position labels, the correct configuration remains ambiguous in 2D. **c** Rotation of the camera immediately reveals the three-dimensional configuration of the fragments. **d** After splicing axonal fragments (magenta segments) and removal of false segmentation results, branches may have to be connected at branching points (one such point is indicated by the arrow). **e** Close-up of the region pointed at by the arrow in d. Nodes are displayed as circles, edge points as small squares. The edge point to be turned into a branching node and the terminal node of the upper edge are selected and (**f**) spliced, resulting in a point-to-node conversion and a new edge connecting the selected node with the new branching node. **g** The reconstruction result superimposed onto the MIP, and (**h**) viewed in 3D. Image and caption taken from [(Dercksen et al., 2014)](https://link.springer.com/article/10.1007/s12021-013-9213-2#citeas).

## 2. Skeleton to morphology reconstruction

The aforementioned methods output a skeleton of a neuron. The next challenge is now to assign dendrite diameters to this skeleton. To do this, we use a rayburst algorithm. This algorithm:
1. Iterates over pixels that belong to the neuron in each slice
2. Finds the nearest brightest pixel, taken to be a referene point in the neuronal structure in the slice
3. Draws various brightness profiles in all directions
4. Calculates the least wide brightness profile. This width is then taken to be the diameter of the structure at that pixel.

We will illustrate this algorithm by first running the entire pipeline on all slices

In [10]:
from dendrite_thickness.thickness import pipeline

# setup the thicnkess pipeline
p = pipeline.ExtractThicknessPipeline()
p.set_output_path(output_folder_path)
p.set_am_paths_by_folder(am_folder_path)  # These amira paths have auto-identified landmarks
p.set_tif_paths_by_folder(tif_folder_path)  # slice images
p.set_hoc_file(hoc_file_path)  # skeletonized .hoc file
p.set_am_to_hoc_transformation_by_landmarkAscii(bijective_points_path)  # not required
p.set_client_for_parallelization(client=I.get_client())

Reading hoc file /gpfs/soma_fs/scratch/meulemeester/project_src/in_silico_framework/getting_started/radii/data/neuron1/hoc/500_GP_WR639_cell_1547_SP5C_checked_RE.hoc
<Client: 'tcp://10.102.2.81:38786' processes=24 threads=24, memory=2.36 TB>


In [ ]:
df = p.run()

---- initialize project ----
---- setup slice objects ----
In threshold: 0.5
--------
Setting up slice:S36_final_downsampled_dendrites_done_zScale_40_aligned_ascii.am
--------
Setting up slice:S31_final_done_Alison_zScale_40.am
--------
Setting up slice:S32_final_done_Alison_zScale_40.am
--------
Setting up slice:S12_final_done_zScale_40.am
--------
Setting up slice:S35_final_downsampled_dendrites_done_zScale_40_aligned_ascii.am
--------
Setting up slice:S14_final_done_Alison_zScale_40.am
--------
Setting up slice:S26_final_done_Alison_zScale_40.am
--------
Setting up slice:S22_final_done_Alison_zScale_40.am
--------
Setting up slice:S25_final_done_Alison_zScale_40.am
--------
Setting up slice:S17_final_done_Alison_zScale_40.am
--------
Setting up slice:S18_final_done_Alison_zScale_40.am
--------
Setting up slice:S34_final_downsampled_dendrites_done_zScale_40_ascii.am
--------
Setting up slice:S35_final_downsampled_dendrites_done_zScale_40_ascii.am
--------
Setting up slice:S21_final_d

The output is written to disk, as it can get quite large. Let's read it in and see what kind of information we got out of this pipeline. Pandas operates lazily, so this should not eat up your RAM as long as you don't operate on the entire dataset.

In [ ]:
from dendrite_thickness.thickness.analysis import get_all_data_output_table
df = get_all_data_output_table(
p.all_slices,
p.default_threshold
)

In [ ]:
df.head()

Note that the z coordinate is relative within the slice. In order to interpret these in a sensible way, we need to infer what the actual z coordinate would be if we were to stack the slices.

In [ ]:
def get_z(df, slice_thickness=50):
    """ Given the pipeline output in pandas.DataFrame format, get the height of all points"""
    slice_n = [float(e.split(I.os.sep)[-1][1:3]) for e in df.slice]
    point_depths = [z + sn*slice_thickness for z, sn in zip(df.z_slice, slice_n)]

    # Slices are from pia to white matter, so low z here means higher up
    # Let's reverse that, so high z does not mean high depth, but rather height (makes more sense for a plot)
    max_z = slice_thickness*max(slice_n)
    point_z = [max_z - e for e in point_depths]
    return point_z

In [ ]:
import plotly.express as px

def get_fig(pipeline):
    """Given a pipeline object that has run, fetch the output data and construct an interactive plotly 3D scatterplot"""
    df = get_all_data_output_table(
        pipeline.all_slices,
        pipeline.default_threshold)
    
    point_z = get_z(df)
    fig = px.scatter_3d(
        x=df.x_slice, 
        y=df.y_slice, 
        z= [max_z - e for e in point_z],
        size=df['min_thickness_0.5'].values.tolist(),
        color=df['min_thickness_0.5'].values.tolist(),
        opacity=0.1
    )

    # tight layout
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=0))
    fig.update_traces(
        marker=dict(line=dict(width=0)))
    return fig

In [ ]:
fig = get_fig(p)
fig.show(renderer="iframe_connected")

The neuron morphology starts to become visible! However, there are a handful of things that need to be fixed:
1. There are a lot of artifacts floating around the neuron that had significant nonzero brightness values in the slice preparation, but are clearly not part of the neuron.
2. Cutting the slice preparation introduces a shearing effect near the boundary of the slices, and the points at every 50 $\mu m$ are misaligned.

Both of these things need to be fixed in either pre- or post-processing in e.g. Amira.

In [ ]:
fig.write_html(I.os.path.join(current_dir, 'static', 'reconstruction.html'), auto_open=True)

In [ ]:
p2 = p
p2.set_am_paths_by_folder(I.os.path.join(DATA_DIR, 'am_analysis'))
p2.set_output_path(I.os.path.join(DATA_DIR, 'output_aligned'))
p2.run()

In [ ]:
df2 = get_all_data_output_table(
p.all_slices,
p.default_threshold
)

In [ ]:
fig2 = get_fig(p)
fig.show(renderer="iframe_connected")